<a href="https://colab.research.google.com/github/Balajivasudevan03/Feature_engineering_01_and-_02/blob/main/Feature_engineering_01_and_02_Code_documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# why is this code nescessary!!!

import warnings
warnings.filterwarnings('ignore')

In [ ]:
data.describe()    # to only the numerical columns!!!

In [ ]:
data.describe(include = ['object'])   ## only the categorical columns only

In [ ]:
cat_cols = data.dtypes =='object'
cat_cols = list(cat_cols[cat_cols].index)   #### give you only the catgorical column names
cat_cols

In [ ]:
num_cols = data.dtypes !='object'
num_cols = list(num_cols[num_cols].index)  ### gives you only the numerical column names
num_cols

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [15,10]   ### kde-subplots for continous variables
data.plot(kind = 'density', subplots = True, layout = (3,3), sharex = False)
plt.show()

In [ ]:
print(data.skew())    ## prints the skewness of the data

In [ ]:
data.kurt()  ## prints the kurtosis of the data

In [ ]:
data['Loan_Status'].value_counts()
target = 'Loan_Status'


plt.rcParams["figure.figsize"] = [6,4]
sns.countplot(data=data, x='Loan_Status')   ### count plot for loan_status
plt.show()

In [ ]:
plt.subplot(121)
sns.histplot(data["ApplicantIncome"], kde=True)

plt.subplot(122)
data["ApplicantIncome"].plot.box(figsize=(16,5))   ### sub-plot for both hist and box plots
plt.show()

In [ ]:
Income_bin = pd.crosstab(data["Income_bin"],data["Loan_Status"], normalize="index")
Income_bin.plot(kind="bar",figsize=(4,4))
plt.xlabel("ApplicantIncome")
plt.ylabel("Percentage")
plt.show()              ### this is another way to plot the bar/dodged plot

In [ ]:
vals_n = pd.crosstab(data["TotalIncome_bin"], data["Loan_Status"], normalize="index")
vals_n.plot(kind="bar",figsize=(4,4)) ## one more way!!

In [ ]:
# saving the dataframe to CSV file
data.to_csv('updated_loan.csv',index=False) ### updating the old ones to the new ones

## Feature---engineering---02

In [ ]:
# how many percentage of data is missing in each column

missing_value = pd.DataFrame({
    'Missing Value': data.isnull().sum(),
    'Percentage': (data.isnull().sum() / len(data))*100
})
missing_value.sort_values(by='Percentage', ascending=False)

In [ ]:
from sklearn.impute import SimpleImputer

SimpleImputer(strategy="mean").fit_transform(a)

SimpleImputer(strategy="most_frequent").fit_transform(a)

SimpleImputer(strategy="constant",fill_value=1000).fit_transform(a)

In [ ]:
# Missing numeric columns

# set of columns
num_missing = ['EMI', 'Loan_Amount_per_year',  'LoanAmount',  'Loan_Amount_Term']

# instance of simple imputer class with a median strategy
median_imputer = SimpleImputer(strategy = 'median')

for col in num_missing:
    data[col] = pd.DataFrame(median_imputer.fit_transform(pd.DataFrame(data[col])))

In [ ]:
cat_missing = ['Gender', 'Married','Dependents']

freq_imputer = SimpleImputer(strategy = 'most_frequent') # mode
for col in cat_missing:
    data[col] = pd.DataFrame(freq_imputer.fit_transform(pd.DataFrame(data[col])))

In [ ]:
data['Income_bin'] = data['Income_bin'].astype('object')
data['TotalIncome_bin'] = data['TotalIncome_bin'].astype('object')

In [ ]:
df_num = data.select_dtypes(include = np.number)

In [ ]:
df_num.drop(['Credit_History','Able_to_pay_EMI'],axis=1,inplace=True)
df_num.head()

In [ ]:
for col in enumerate(df_num):
    sns.boxplot(x=col[1],data=df_num)
    plt.show()

In [ ]:
# obtain the first quartile
Q1 = df_num.quantile(0.25)

# obtain the third quartile
Q3 = df_num.quantile(0.75)

# obtain the IQR
IQR = Q3 - Q1

# print the IQR
print(IQR)

In [ ]:
df_iqr=data[~((df_num < (Q1-1.5*IQR))|(df_num > (Q3 + 1.5*IQR))).any(axis=1)]  ### removing the outliers

In [ ]:
import scipy
from scipy import stats

# z-scores are defined for each observation in a variable
# compute the z-scores using the method zscore from the scipy library
z_scores_LoanAmount = scipy.stats.zscore(df_num["LoanAmount"])

# display the z-scores
z_scores_LoanAmount

In [ ]:
# print the rows where the z-score is less than -3
row_index_less = np.where(z_scores_LoanAmount < -3)

# print the values
print(row_index_less)

(array([], dtype=int64),)


In [ ]:
# print the rows where the z-score is more than 3
row_index_more = np.where(z_scores_LoanAmount > 3)

# print the values more than the
print(row_index_more)

(array([130, 155, 171, 177, 278, 308, 333, 369, 432, 487, 506, 523, 525,
       561, 604]),)


In [ ]:
# count of outliers in the variable representing LoanAmount
len(row_index_less[0]) + len(row_index_more[0])

15

In [ ]:
# Filter out the outlier values
# ~: Select all rows which do not satisfy the condition
df_LoanAmount_zscore = data["LoanAmount"][~(( z_scores_LoanAmount < -3) |(z_scores_LoanAmount > 3))]

In [ ]:
z_score_threshold = 3  # You can adjust this threshold as needed

# Calculate z-scores for all numerical columns
z_scores = np.abs(stats.zscore(data.select_dtypes(include=[np.number])))

# Remove rows with any z-score exceeding the threshold
data_no_outliers = data[(z_scores < z_score_threshold).all(axis=1)]

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
col='Loan_Status'
label_encoder=LabelEncoder()
df_iqr[col]=label_encoder.fit_transform(df_iqr[col])
df_iqr[col].value_counts()

In [ ]:
col="Gender"
df_iqr[col]=label_encoder.fit_transform(df_iqr[col])
df_iqr[col].value_counts()

col="Married"
df_iqr[col]=label_encoder.fit_transform(df_iqr[col])
df_iqr[col].value_counts()

col="Education"
df_iqr[col]=label_encoder.fit_transform(df_iqr[col])
df_iqr[col].value_counts()

In [ ]:
from category_encoders import TargetEncoder

In [ ]:
pd.crosstab(df_iqr["Property_Area"], df_iqr["Loan_Status"])

pd.crosstab(df_iqr["Property_Area"], df_iqr["Loan_Status"],normalize="index")


col="Property_Area"
te=TargetEncoder()

# here I need to pass 2 things as parameters. 1st the column that I'm working on
# 2nd the Target column

df_iqr[col]=te.fit_transform(df_iqr[col],df_iqr["Loan_Status"])
df_iqr[col].value_counts()



col="Self_Employed"
te=TargetEncoder()
df_iqr[col]=te.fit_transform(df_iqr[col],df_iqr["Loan_Status"])
df_iqr[col].value_counts()

col="Dependents"
te=TargetEncoder()
df_iqr[col]=te.fit_transform(df_iqr[col],df_iqr["Loan_Status"])
df_iqr[col].value_counts()

col="Income_bin"
te=TargetEncoder()
df_iqr[col]=te.fit_transform(df_iqr[col],df_iqr["Loan_Status"])
df_iqr[col].value_counts()

col="TotalIncome_bin"
te=TargetEncoder()
df_iqr[col]=te.fit_transform(df_iqr[col],df_iqr["Loan_Status"])
df_iqr[col].value_counts()

In [ ]:
# import StandardScaler
from sklearn.preprocessing import StandardScaler

# instantiate the standardscaler
standard_scale = StandardScaler()

# fit the StandardScaler
num['Scaled_LoanAmount'] = standard_scale.fit_transform(num[['LoanAmount']])

# calculate the minimum and maximum values of the variable
print(" The minimum value of the sales:",num['Scaled_LoanAmount'].min(),"\n",
      "The maximum value of the sales:", num['Scaled_LoanAmount'].max())

In [ ]:
# set the figure size
plt.rcParams["figure.figsize"] = [15,8]

# subplot is used to create a set of plots
# we create a subplot of 1 row by 2 columns
# use first column for plotting
plt.subplot(1,2,1)

# plot the distribution of the original variable
# axlabel: label the x-axis
sns.distplot(num['LoanAmount'])
# set label for the y-axis
plt.ylabel('Density', fontsize=15)
# set label for x-axis
plt.xlabel('LoanAmount', fontsize=15)

# We create a subplot of 1 row by 2 columns
# use the second column for plotting
plt.subplot(1,2,2)

# plot the distribution of the scaled variable
# axlabel: label the x-axis
sns.distplot(num['Scaled_LoanAmount'])
# set label for the y-axis
plt.ylabel('Density', fontsize=15)
# set label for the x-axis
plt.xlabel('Scaled LoanAmount', fontsize=15)
# display the plot
plt.show()

In [ ]:
# import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# instantiate the MinMaxScaler
min_max = MinMaxScaler()

# fit the MinMaxScaler
num['minmax_LoanAmount'] = min_max.fit_transform(num[['LoanAmount']])

# minimum and maximum value of the normalized variable
num['minmax_LoanAmount'].min(), num['minmax_LoanAmount'].max()

In [ ]:
# set the figure size
plt.rcParams["figure.figsize"] = [15,8]

# subplot is used to create a set of plots
# We create a subplot of 1 row by 2 columns
# use the first column for plotting
plt.subplot(1,2,1)

# plot the distribution of the original variable
# axlabel: label the x-axis
sns.distplot(num['LoanAmount'], axlabel='LoanAmount')
# set label for the y-axis
plt.ylabel('Density', fontsize=15)
# set label for the x-axis
plt.xlabel('LoanAmount', fontsize=15)

# we create a subplot of 1 row by 2 columns
# use the second column for plotting
plt.subplot(1,2,2)

# plot the distribution of the scaled variable
# axlabel: label the x-axis
sns.distplot(num['minmax_LoanAmount'], axlabel='Scaled LoanAmount')
# set label for the y-axis
plt.ylabel('Density', fontsize=15)
# set label for the x-axis
plt.xlabel('Scaled LoanAmount', fontsize=15)

# display the plot
plt.show()